### Vector stores and retrievers
This video tutorial will familiarize you with LangChain's vector store and retriever abstractions. These abstractions are designed to support retrieval of data-- from (vector) databases and other sources-- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.

We will cover 
- Documents
- Vector stores
- Retrievers


In [18]:
!pip install langchain
!pip install langchain-chroma
!pip install langchain_groq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Documents
LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content;
- metadata: a dict containing arbitrary metadata.
The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

Let's generate some sample documents:

In [19]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [20]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [21]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm=ChatGroq(groq_api_key=groq_api_key,model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x3103daa20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x162ed1880>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [22]:
!pip install langchain_huggingface

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [24]:
## VectorStores
from langchain_chroma import Chroma

vectorstore=Chroma.from_documents(documents,embedding=embeddings)
vectorstore


In [25]:
vectorstore.similarity_search("cat")

[Document(id='6776be69-37a7-446b-a1d9-d5c3353511ce', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='5c22bfcb-f5bb-4060-9e67-3883aa9ae4a0', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='8c6bbc8f-8bc3-4260-b54f-be17a13c9d80', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='2c7b0d61-9ae2-4c64-93aa-83c57c71c425', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [26]:
# Async query: await is used in asynchronous programming to call
# another async function(asimilarity_search) and waits for it
# a in asimilarity_search() stands for "asynchronous".
await vectorstore.asimilarity_search("cat")

# async def fetch_data() → defines an async function.
# await get_data_from_api() → calls another async function and waits for it.

'''
import asyncio

# Simulate boiling water
async def boil_water():
    print("Boiling water...")       # Output: Boiling water...
    await asyncio.sleep(2)
    print("Water is ready!")        # Output: Water is ready! (after 2 seconds)

# Simulate toasting bread
async def toast_bread():
    print("Toasting bread...")      # Output: Toasting bread...
    await asyncio.sleep(3)
    print("Bread is toasted!")      # Output: Bread is toasted! (after 3 seconds)

# Main function to run both tasks
async def make_breakfast():
    await asyncio.gather(
        boil_water(),
        toast_bread()
    )
    print("Breakfast is ready!")    # Output: Breakfast is ready! (after both tasks finish)

# Run the async function
asyncio.run(make_breakfast())

# Final Output Timeline:
# Boiling water...
# Toasting bread...
# Water is ready!         (after 2 seconds)
# Bread is toasted!       (after 3 seconds)
# Breakfast is ready!     (after both are done)
'''

'\nimport asyncio\n\n# Simulate boiling water\nasync def boil_water():\n    print("Boiling water...")       # Output: Boiling water...\n    await asyncio.sleep(2)\n    print("Water is ready!")        # Output: Water is ready! (after 2 seconds)\n\n# Simulate toasting bread\nasync def toast_bread():\n    print("Toasting bread...")      # Output: Toasting bread...\n    await asyncio.sleep(3)\n    print("Bread is toasted!")      # Output: Bread is toasted! (after 3 seconds)\n\n# Main function to run both tasks\nasync def make_breakfast():\n    await asyncio.gather(\n        boil_water(),\n        toast_bread()\n    )\n    print("Breakfast is ready!")    # Output: Breakfast is ready! (after both tasks finish)\n\n# Run the async function\nasyncio.run(make_breakfast())\n\n# Final Output Timeline:\n# Boiling water...\n# Toasting bread...\n# Water is ready!         (after 2 seconds)\n# Bread is toasted!       (after 3 seconds)\n# Breakfast is ready!     (after both are done)\n'

In [27]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='6776be69-37a7-446b-a1d9-d5c3353511ce', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351056814193726),
 (Document(id='5c22bfcb-f5bb-4060-9e67-3883aa9ae4a0', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351056814193726),
 (Document(id='8c6bbc8f-8bc3-4260-b54f-be17a13c9d80', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.574089765548706),
 (Document(id='2c7b0d61-9ae2-4c64-93aa-83c57c71c425', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.574089765548706)]

### Retrievers
LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
# RunnableLambda: It turns a regular Python function into something LangChain can run as part of a LCEL chain.
# Supports .invoke(), .batch(), streaming, etc.
# bind(k=1): Number of documents to retrieve.
retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
# batch(): Processes multiple inputs in one go. invoke() is for single input.
retriever.batch(["cat","dog"])

[[Document(id='5c22bfcb-f5bb-4060-9e67-3883aa9ae4a0', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='2c7b0d61-9ae2-4c64-93aa-83c57c71c425', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [ ]:
retriever=vectorstore.as_retriever(
    search_type="similarity",
    # "k":1 tells the retrieve to return 1 document per query.
    search_kwargs={"k":1}
)
retriever.batch(["cat","dog"])


[[Document(id='5c22bfcb-f5bb-4060-9e67-3883aa9ae4a0', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='2c7b0d61-9ae2-4c64-93aa-83c57c71c425', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [30]:
## RAG
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])
# RunnablePassthrough(): Forwards input without changes. Basically we are using it to pass the question as it is.
# When we invoke the rag_chain later, it will replace {question} in the prompt with the input question.
# The retriever will fetch relevant documents for {context}.
# So the final prompt will have both the question and the context filled in.
rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm

response=rag_chain.invoke("tell me about dogs")
print(response.content)
response


Dogs are great companions, known for their loyalty and friendliness.


AIMessage(content='Dogs are great companions, known for their loyalty and friendliness.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 112, 'total_tokens': 127, 'completion_time': 0.010889165, 'prompt_time': 0.005949841, 'queue_time': 0.052389308, 'total_time': 0.016839006}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e32974efee', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--dc915147-02cb-448a-b1e5-5658f56079f4-0', usage_metadata={'input_tokens': 112, 'output_tokens': 15, 'total_tokens': 127})